In [1]:
import io
import os
import sys
import csv
import gzip
import time
import json
import secrets
import multiprocessing
import subprocess
import resource
import pandas as pd
# import sqlite3
# from PIL import Image
# import matplotlib.pyplot as plt
# from plotnine import *
import numpy as np
import scipy as sp
import statsmodels.api as sm
import random
# from collections import Counter
# import seaborn as sns
# import matplotlib.colors as mcolors
# from matplotlib.ticker import FuncFormatter
import itertools
import collections
# import patchworklib as pw
# sys.path.append('/well/band/users/rbx225/software/lcwgsus/')
# import lcwgsus
# from lcwgsus.variables import *
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

pd.options.mode.chained_assignment = None

In [43]:
gene = 'DRB1'
ipd_gen_file = '/Users/sus_zhang/Desktop/Suuuuuuuus/misc_data/alignments/DRB1_gen.txt'


In [9]:
with open(ipd_gen_file, "r") as file:
    lines = file.readlines()
lines

['# file: DRB1_gen.txt\n',
 '# date: 2020-01-20\n',
 '# version: IPD-IMGT/HLA 3.39.0\n',
 '# origin: http://hla.alleles.org/wmda/DRB1_gen.txt\n',
 '# repository: https://raw.githubusercontent.com/ANHIG/IMGTHLA/Latest/alignments/DRB1_gen.txt\n',
 '# author: WHO, Steven G. E. Marsh (steven.marsh@ucl.ac.uk)\n',
 '                   \n',
 ' gDNA              -599\n',
 '                   |\n',
 ' DRB1*01:02:01:01  GCATCCACAG AATCACA... TTTTCTAGTG TTGAAAGACC TGAAAGATCA CGGTGCCTTC ATTTCAA.CT GTGAGACATG AAGTAATTTT CCCAAATCTA \n',
 ' DRB1*01:02:01:02  ********** ********** ********** ********** ********** ********** ********** ********** ********** ********** \n',
 ' DRB1*01:01:01:01  ********** ********** ********** **-------- ---------- ---------- -------.-- ---------- ---------- ---------- \n',
 ' DRB1*01:01:01:02  ********** ********** ********** ********** ********** ********** ********** ********** ********** ********** \n',
 ' DRB1*01:01:34     ********** ********** ********** *********

In [52]:
gDNA_idx = []
names = []
i = 0
while len(gDNA_idx) < 2:
    l = lines[i]
    if 'gDNA' in l:
        gDNA_idx.append(i)
    elif l.lstrip(' ').split(' ')[0].startswith(gene + '*'):
        name = l.lstrip(' ').split(' ')[0]
        names.append(name)
    i += 1

In [21]:
first_base = int(lines[gDNA_idx[0]].split(' ')[-1].split('\n')[0])
n_alleles = gDNA_idx[1] - gDNA_idx[0] - 3


In [55]:
alleles_dict = {k:'' for k in names}
for i, s in enumerate(lines):
    r = s.lstrip(' ')
    if r.startswith(gene):
        r = r.rstrip(' \n')
        name = r.split(' ')[0]
        sequence = r.split(' ')[2:]
        sequence = ''.join(sequence)
        alleles_dict[name] = alleles_dict[name] + sequence
    

In [166]:
df = pd.DataFrame({key: list(value) for key, value in alleles_dict.items()}).T
df = df.drop(columns=df.columns[df.eq('|').all()])
df.columns = range(df.shape[1])

length = len(df.columns)
positions = [first_base]*length

df.loc[len(df)] = positions

r_idx = len(df) - 1
for i in df.columns[1:]:
    if df.iloc[0, i] == '.':
        df.iloc[r_idx, i] = df.iloc[r_idx, i-1]
    else:
        if df.iloc[r_idx, i-1] != -1:
            df.iloc[r_idx, i] = df.iloc[r_idx, i-1] + 1
        else:
            df.iloc[r_idx, i] = df.iloc[r_idx, i-1] + 2 # No zero in position. ATG is encoded by 1.
df.columns = df.iloc[r_idx]
df = df.iloc[:r_idx]

In [119]:
df.iloc[1,-15]

'*'

In [ ]:
o.iloc[r_idx]

In [100]:
o

,0,1,2,3,4,5,6,7,8,9,...,18471,18472,18473,18474,18475,18476,18477,18478,18479,18480
DRB1*01:02:01:01,G,C,A,T,C,C,A,C,A,G,...,T,G,G,A,G,T,T,A,A,A
DRB1*01:02:01:02,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
DRB1*01:01:01:01,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
DRB1*01:01:01:02,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
DRB1*01:01:34,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DRB1*16:02:01:01,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
DRB1*16:02:01:02,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
DRB1*16:02:01:03,*,*,*,*,*,*,*,*,*,-,...,*,*,*,*,*,*,*,*,*,*
DRB1*16:02:01:04,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*


In [53]:
names

['DRB1*01:02:01:01',
 'DRB1*01:02:01:02',
 'DRB1*01:01:01:01',
 'DRB1*01:01:01:02',
 'DRB1*01:01:34',
 'DRB1*01:03:01',
 'DRB1*03:01:01:01',
 'DRB1*03:01:01:02',
 'DRB1*03:01:01:03',
 'DRB1*03:02:01',
 'DRB1*04:01:01:01',
 'DRB1*04:01:01:02',
 'DRB1*04:01:01:03',
 'DRB1*04:02:01',
 'DRB1*04:03:01:01',
 'DRB1*04:03:01:02',
 'DRB1*04:04:01',
 'DRB1*04:05:01:01',
 'DRB1*04:05:01:02',
 'DRB1*04:05:01:03',
 'DRB1*04:05:01:04',
 'DRB1*04:06:01',
 'DRB1*04:07:01:01',
 'DRB1*04:07:01:02',
 'DRB1*04:10:01',
 'DRB1*04:10:03',
 'DRB1*04:11:01',
 'DRB1*04:12',
 'DRB1*07:01:01:01',
 'DRB1*07:01:01:02',
 'DRB1*07:01:01:03',
 'DRB1*07:01:01:04',
 'DRB1*08:02:01:01',
 'DRB1*08:02:01:02',
 'DRB1*08:03:02:01',
 'DRB1*08:03:02:02',
 'DRB1*08:03:02:03',
 'DRB1*08:09:01',
 'DRB1*09:01:02:01',
 'DRB1*09:01:02:02',
 'DRB1*09:01:02:03',
 'DRB1*09:01:02:04',
 'DRB1*09:21',
 'DRB1*09:32',
 'DRB1*10:01:01:01',
 'DRB1*10:01:01:02',
 'DRB1*10:01:01:03',
 'DRB1*11:01:01:01',
 'DRB1*11:01:01:02',
 'DRB1*11:01:01:03'